### Creating Connection

In [1]:
import mysql.connector as mysql
import pandas as pd
try:

    mydb = mysql.connect(host='localhost', user='root',
                         password='mypass', auth_plugin='mysql_native_password')

except Exception as e:

    print(
        'Failed to create connection with mysql server, reason {0}'.format(e))

### Creating Cursor

In [2]:
try:

    cursor = mydb.cursor(buffered=True)

except Exception as e:

    print('Error occur while creating cursor object, reason {0}'.format(e))

### Creating Database and Schema

In [3]:
def create_database_schema():
    cursor.execute("DROP DATABASE IF EXISTS RandomX")
    cursor.execute("CREATE DATABASE RandomX")
    cursor.execute("USE RandomX")
    
    cursor.execute("""
        CREATE TABLE users(
            username VARCHAR (50),
            gender ENUM("m", "f") NOT NULL,
            age INTEGER NOT NULL,
            PRIMARY KEY (username)
        )""")
    cursor.execute("""
        CREATE TABLE movies(
            movie_id VARCHAR (255),
            movie_title VARCHAR (255) NOT NULL,
            audience_rating FLOAT(4,2) NOT NULL,
            PRIMARY KEY (movie_id)
        )""")
    cursor.execute("""
        CREATE TABLE movie_reviews(
            movie_id VARCHAR (255) NOT NULL, 
            username VARCHAR (50) NOT NULL,
            movie_rev VARCHAR (500) NOT NULL,
            FOREIGN KEY (movie_id) REFERENCES movies(movie_id),
            FOREIGN KEY (username) REFERENCES users(username)
        )""")
    mydb.commit()

In [4]:
create_database_schema()

### Inserting Dummy Records

In [5]:
def insert_user(username, gender, age):
    sql_query = "INSERT INTO users VALUES (%s, %s, %s)"
    values = (username, gender, age)
    cursor.execute(sql_query, values)
    mydb.commit()

def insert_movie(movie_id, movie_title, audience_rating):
    sql_query = "INSERT INTO movies VALUES (%s, %s, %s)"
    values = (movie_id, movie_title, audience_rating)
    cursor.execute(sql_query, values)
    mydb.commit()
    
def insert_movie_review(movie_id, username, movie_rev):
    sql_query = "INSERT IGNORE INTO movie_reviews VALUES (%s, %s, %s)"
    values = (movie_id, username, movie_rev)
    cursor.execute(sql_query, values)
    mydb.commit()

In [6]:
user_df = pd.read_excel('userdata.xls')
user_df

,username,gender,age
0,leyclema,m,28
1,carchery,f,33
2,hockeren,m,32
3,reoloque,f,26
4,monsit,m,29
5,ivartherf,m,25
6,icap,f,37
7,sentact,f,39
8,andwrol,f,32
9,oryoustald,m,26


In [7]:
for index, row in user_df.iterrows():
    insert_user(row['username'], row['gender'], row['age'])

In [8]:
cursor.execute("SELECT * FROM users")
output_list = cursor.fetchall()
for row in output_list:
    print(*row)

aching f 39
alfa m 29
andwrol f 32
carchery f 33
eryptite f 30
gambo m 36
gamma m 27
hockeren m 32
icap f 37
inkag m 35
ivartherf m 25
leyclema m 28
monsit m 29
nelate f 39
oryoustald m 26
patisha f 22
porpsti m 32
reoloque f 26
sentact f 39
urunorydr m 33


In [9]:
movie_df = pd.read_excel(
     "movie_info.xlsx",
     engine='openpyxl'
)
movie_df

,movie_id,movie_title,audience_rating
0,m/the_two_popes,The Two Popes,4.40
1,m/the_lord_of_the_rings_the_fellowship_of_the_...,The Lord of the Rings: The Fellowship of the Ring,4.75
2,m/the_signal_2014,The Signal,1.90
3,m/up_in_the_air_2009,Up in the Air,3.95
4,m/sphere,Sphere,1.90
...,...,...,...
95,m/zookeeper,Zookeeper,2.05
96,m/silent_light,Silent Light (Stellet licht),3.45
97,m/1171939-perfect_stranger,Perfect Stranger,2.55
98,m/block_party,Dave Chappelle's Block Party,3.70


In [10]:
for index, row in movie_df.iterrows():
    insert_movie(row['movie_id'], row['movie_title'], row['audience_rating'])

In [11]:
cursor.execute("SELECT * FROM movies")
output_list = cursor.fetchall()
for row in output_list:
    print(*row)

m/10004925-matador The Matador 3.25
m/10006588-street_fight Street Fight 4.65
m/10009169-man_in_the_chair Man in the Chair 3.4
m/10009254-shutter Shutter 1.65
m/10009274-priest Priest 2.3
m/1023543-white_christmas White Christmas 4.4
m/1070608-quest The Quest 1.8
m/1079818-anastasia Anastasia 3.85
m/1083535-mafia Mafia! 2.35
m/1093579-man_on_the_moon Man on the Moon 3.95
m/1113375-showtime Showtime 1.25
m/1153228-the_ringer The Ringer 3.4
m/1171939-perfect_stranger Perfect Stranger 2.55
m/1193303-1193303-trumbo Trumbo 3.9
m/1197120-oss_117_cairo_nest_of_spies OSS 117: Le Caire Nid d'Espions (OSS 117: Cairo, Nest of Spies) 3.75
m/1210830-antichrist Antichrist 2.75
m/1218217-single_man A Single Man 4.05
m/a_guide_to_recognizing_your_saints A Guide to Recognizing Your Saints 3.65
m/american_werewolf_in_london An American Werewolf in London 4.25
m/avengement Avengement 3.25
m/bad_education Bad Education 4.3
m/big_hit The Big Hit 2.85
m/black_or_white Black or White 3.25
m/block_party Dave 

In [12]:
movie_review_df = pd.read_excel('movie_reviews.xls')
movie_review_df

,movie_id,user_name,movie_rev
0,m/the_two_popes,gamma,Jonathan Pryce and Anthony Hopkins engage in a...
1,m/the_two_popes,hockeren,Somehow the filmmakers found lightheartedness ...
2,m/the_two_popes,carchery,There's an unquestionable appeal to the way th...
3,m/the_two_popes,sentact,Despite its over reliance on unnecessary flash...
4,m/the_two_popes,oryoustald,Hopkins and Pryce's finely tuned performances ...
...,...,...,...
495,m/the_double_2013,nelate,...not only does The Double confirm Ayoade as ...
496,m/the_double_2013,gambo,Ayoade's care with the movie's craft is loving...
497,m/the_double_2013,hockeren,"The Double mirrors aspects of Gilliam, Gondry ..."
498,m/the_double_2013,ivartherf,"This is paranoia played for laughs, with a ver..."


In [13]:
for index, row in movie_review_df.iterrows():
    insert_movie_review(row['movie_id'], row['user_name'], row['movie_rev'])

In [14]:
cursor.execute("SELECT * FROM movie_reviews")
output_list = cursor.fetchall()
for row in output_list:
    print(*row)

m/the_two_popes gamma Jonathan Pryce and Anthony Hopkins engage in a theological joust, equal parts levity, humor, and spiritual exploration. There is no resolution of millennia-old debates, just amusing exposition and proselytizing
m/the_two_popes hockeren Somehow the filmmakers found lightheartedness and - gasp - laughs in a story of political intrigue at the top of the notoriously buttoned-up Catholic Church.
m/the_two_popes carchery There's an unquestionable appeal to the way the movie transforms a weighty and divisive topic into more approachable terms.
m/the_two_popes sentact Despite its over reliance on unnecessary flashbacks, "The Two Popes" is a masterclass in acting delivered by two legendary stalwarts of the craft: Pryce and Hopkins.
m/the_two_popes oryoustald Hopkins and Pryce's finely tuned performances illuminate Benedict's shrewd intelligence and Francis's deep humility.
m/the_lord_of_the_rings_the_fellowship_of_the_ring ivartherf You think Harry Potter had expectations?

m/the_prophet_2014 hockeren ...like surveying the desert cart at a nice restaurant...part of the experience is the fact that so many different (and delicious) treats are on offer at once.
m/my_darling_clementine nelate My Darling Clementine must be one of the sweetest and most good-hearted of all Westerns.
m/my_darling_clementine reoloque ...a relatively laid-back, almost poetic vision of the Old West, its characters and characterizations outshining its well-worn plot.
m/my_darling_clementine nelate The reason My Darling Clementine never seems to stray or lose focus is Henry Fonda
m/my_darling_clementine carchery The best Western John Ford ever made without John Wayne on board.
m/my_darling_clementine eryptite I haven't seen a better docudrama about Wyatt Earp, but on the other hand, the venerated lawman has yet to be the subject of a masterpiece
m/little_children reoloque Field offers a darkly humorous view of American suburbanites whose paths crisscross in unexpected ways. It's a cla

### Add Column 'Sentiment'

In [15]:
positive_text_file = open('positive_words.txt', 'r')
positive_list = positive_text_file.read().splitlines()
positive_list

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation',
 'accolade',
 'accolades',
 'accommodative',
 'accomodative',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accomplishments',
 'accurate',
 'accurately',
 'achievable',
 'achievement',
 'achievements',
 'achievible',
 'acumen',
 'adaptable',
 'adaptive',
 'adequate',
 'adjustable',
 'admirable',
 'admirably',
 'admiration',
 'admire',
 'admirer',
 'admiring',
 'admiringly',
 'adorable',
 'adore',
 'adored',
 'adorer',
 'adoring',
 'adoringly',
 'adroit',
 'adroitly',
 'adulate',
 'adulation',
 'adulatory',
 'advanced',
 'advantage',
 'advantageous',
 'advantageously',
 'advantages',
 'adventuresome',
 'adventurous',
 'advocate',
 'advocated',
 'advocates',
 'affability',
 'affable',
 'affably',
 'affectation',
 'affection',
 'affectionate',
 'affinity',
 'affirm',
 'affirmation',
 'affirmative',
 'affluence',
 'affluent',
 'afford',
 'affordable',
 'af

In [16]:
negative_text_file = open('negative_words.txt', 'r')
negative_list = negative_text_file.read().splitlines()
negative_list

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted',
 'aborts',
 'abrade',
 'abrasive',
 'abrupt',
 'abruptly',
 'abscond',
 'absence',
 'absent-minded',
 'absentee',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abused',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accost',
 'accursed',
 'accusation',
 'accusations',
 'accuse',
 'accuses',
 'accusing',
 'accusingly',
 'acerbate',
 'acerbic',
 'acerbically',
 'ache',
 'ached',
 'aches',
 'achey',
 'aching',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addicted',
 'addicting',
 'addicts',
 'admonish',
 'admonisher',
 'admonishingly',
 'admonishment',
 'admonition',
 'adulterate',
 'adulterated',
 'adulteration',
 'adulterier',
 'adversarial',
 'adversary',
 'adverse',
 'adversity',
 'afflict',
 'affliction',
 'afflictive',
 'affront',


In [17]:
def alter_table_add_column_sentiment():
    cursor.execute("""ALTER TABLE movie_reviews
                    ADD sentiment ENUM('positive', 'negative', 'neutral') NOT NULL DEFAULT 'neutral'
                    """)
    mydb.commit()

In [18]:
alter_table_add_column_sentiment()

In [19]:
def get_movie_sentiment(movie_rev):
    sentiment_score = 0
    words = movie_rev.split()
    for word in words:
        if word in positive_list:
            sentiment_score += 1
        elif word in negative_list:
            sentiment_score -= 1
    if sentiment_score > 0:
        return 'positive'
    elif sentiment_score < 0:
        return 'negative'
    return 'neutral'

In [20]:
def update_movie_setiment(movie_id, username, movie_rev):
    movie_sentiment = get_movie_sentiment(movie_rev)
    sql_query = """
        UPDATE movie_reviews
        SET sentiment = %s
        WHERE movie_id = %s AND username = %s AND movie_rev = %s
        """
    values = (movie_sentiment, movie_id, username, movie_rev)
    cursor.execute(sql_query, values)
    mydb.commit()

In [21]:
cursor.execute("SELECT * FROM movie_reviews")
output_list = cursor.fetchall()
for row in output_list:
    movie_id, username, movie_rev = row[:3]
    update_movie_setiment(movie_id, username, movie_rev)

In [22]:
cursor.execute("SELECT * FROM movie_reviews")
output_list = cursor.fetchall()
for row in output_list:
    print(*row)

m/the_two_popes gamma Jonathan Pryce and Anthony Hopkins engage in a theological joust, equal parts levity, humor, and spiritual exploration. There is no resolution of millennia-old debates, just amusing exposition and proselytizing positive
m/the_two_popes hockeren Somehow the filmmakers found lightheartedness and - gasp - laughs in a story of political intrigue at the top of the notoriously buttoned-up Catholic Church. neutral
m/the_two_popes carchery There's an unquestionable appeal to the way the movie transforms a weighty and divisive topic into more approachable terms. positive
m/the_two_popes sentact Despite its over reliance on unnecessary flashbacks, "The Two Popes" is a masterclass in acting delivered by two legendary stalwarts of the craft: Pryce and Hopkins. neutral
m/the_two_popes oryoustald Hopkins and Pryce's finely tuned performances illuminate Benedict's shrewd intelligence and Francis's deep humility. positive
m/the_lord_of_the_rings_the_fellowship_of_the_ring ivarthe

m/room_2015 carchery Abrahamson seamlessly translates Donaghue's work into cinematic terms. positive
m/room_2015 sentact Lenny Abrahamson guides Jack Temblay to the finest performance by a young actor since Quvenzhané Wallis in "Beasts of the Southern Wild." positive
m/room_2015 gambo If Abrahamson were as gifted with a camera as he was with his cast (he inspires subtlety even from the tiny Tremblay), Room could have been truly worthy of the astonishing performances that provide its foundation. positive
m/wings monsit Admittedly, this movie has not aged as well as many of its contemporaries, but it's not hard to understand why it was accorded the Oscar (a term that, by the way, had not yet been coined). neutral
m/wings porpsti The strengths of the film are the great battle sequences ... and the shining moments when the beautiful Clara Bow is onscreen. positive
m/wings aching Looked at freshly, it's surprising that a lot of its best moments aren't the much-celebrated airborne ones, but 

### Fetching Top N movies above audience rating and positive sentiments

In [23]:
def fetch_top_n_movies_audience_rating(n, audience_rating, positive_sentiments):
    cursor.execute("""
        SELECT movie_id, movie_title, audience_rating, COUNT(*) AS 'positive_reviews_count'
        FROM movies NATURAL JOIN movie_reviews
        WHERE sentiment = 'positive'
        GROUP BY movie_id, movie_title, audience_rating
        HAVING COUNT(*) >= %s AND audience_rating > %s
        ORDER BY audience_rating DESC
        LIMIT %s
        """, (positive_sentiments, audience_rating, n) )
    output_list = cursor.fetchall()
    for row in output_list:
        print(*row)

In [24]:
fetch_top_n_movies_audience_rating(5, 3.50, 2)

m/the_lord_of_the_rings_the_fellowship_of_the_ring The Lord of the Rings: The Fellowship of the Ring 4.75 3
m/room_2015 Room 4.65 5
m/man_who_shot_liberty_valance The Man Who Shot Liberty Valance 4.6 3
m/run_lola_run Run Lola Run 4.5 2
m/star_trek_into_darkness Star Trek Into Darkness 4.45 5
